In [92]:
import sys, os, math, time
sys.path.append("..")

import outputDataAggregator as agg
import dataVisualizer as vis
import myStructure as my
import adabooster as myBoost
import globalStuff as glbl
import pandas as pd, numpy as np, 


SyntaxError: invalid syntax (1104226353.py, line 9)

In [34]:
base_dir = base_dir = os.getcwd()  # Gets the directory where your script is
print(base_dir)

/Users/joedngangmeni/Documents/school/Research/RHF_Share/RHFnGAN/Code/AdaBoost


In [35]:
focusParentDir, focusDataDir, MAX_RUNS, ESTNUM, DEPTH, topNUM = my.paramDecider('ADA')

You entered 'yes'. Beginning test run sequence...



In [36]:
rawDataPath = os.path.abspath(os.path.join(base_dir, '..', '..', f'{focusParentDir}', 'RawData', f'{focusDataDir}'))
aggDataPath = os.path.abspath(os.path.join(base_dir, '..', '..', f'{focusParentDir}', 'AggData',f'{focusDataDir}'))
graphsPath = os.path.abspath(os.path.join(base_dir, '..', '..', f'{focusParentDir}', 'Graphs',f'{focusDataDir}'))
tablesPath = os.path.abspath(os.path.join(base_dir, '..', '..', f'{focusParentDir}', 'Tables',f'{focusDataDir}'))

In [88]:
import inputDataParser as parse
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
X, y = parse.getAirData()

In [38]:
NUMTREES = 5
DEPTH = 3

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, shuffle=True)

base_estimator = DecisionTreeRegressor(max_depth=DEPTH)

ada = AdaBoostRegressor(estimator=base_estimator, n_estimators=NUMTREES)

# print(f'X_train head:\t{X_train.columns}\ny_train head:\t{y_train}\nX_test head:\t{X_test.columns}\ny_test head:\t{y_test}\n')
# Train the model
ada.fit(X_train, y_train)

AdaBoostRegressor(estimator=DecisionTreeRegressor(max_depth=3), n_estimators=5)

In [79]:
def regBoostingStats(data: pd.DataFrame, answers: np.array):
    staged_predictions = ada.staged_predict(data)

    r2, mse, rmse, mae = [], [], [], []
    for stage, y_pred in enumerate(staged_predictions):
        # Calculate R2
        
        r2.append(r2_score(answers, y_pred))
        
        # Calculate MSE
        mse.append(mean_squared_error(answers, y_pred))

        # Calculate MAE
        mae.append(mean_absolute_error(answers, y_pred))
    
    for num in mse:
        rmse.append(math.sqrt(num))
    
    return r2, rmse, mse, mae 
        


In [87]:
r2_train, rmse_train, mse_train, mae_train = regBoostingStats(X_train, y_train)
r2_test, rmse_test, mse_test, mae_test = regBoostingStats(X_test, y_test)


print(mse_train)
print(rmse_train)
print(mse_test)
print(rmse_test)

[23523.388885143224, 23523.388885143224, 21533.371624352974, 21530.91158994749, 20829.376469963183]
[153.37336432752338, 153.37336432752338, 146.742535157169, 146.73415277278664, 144.32385966971358]
[22880.703553101746, 22880.703553101746, 20330.074016495513, 21045.07127304066, 20684.7247621654]
[151.26368881229146, 151.26368881229146, 142.5835685361238, 145.06919477628824, 143.8218507813239]


In [89]:
def clsBoostingStats(data: pd.DataFrame, answers: np.array):
    staged_predictions = ada.staged_predict(data)

    mlogloss, f1, accuracy, precision, recall  = [], [], [], [], []

    for stage, y_pred in enumerate(staged_predictions):
        f1.append(f1_score(answers, y_pred, average='weighted' , zero_division=0))
        accuracy.append(accuracy_score(answers, y_pred))
        precision.append(precision_score(answers, y_pred, average="weighted", zero_division=0))
        recall.append(recall_score(answers, y_pred, average='weighted', zero_division=0))

        # Calculate multi-logloss
        y_pred_proba = ada.staged_predict_proba(data)  
        mlogloss.append(log_loss(answers, y_pred_proba, labels=ada.classes_))  

    
    return f1, accuracy, precision, recall, mlogloss

In [98]:
Y, z = parse.getHARData()

Z_train, Z_test, j_train, j_test = train_test_split(Y, z, test_size=0.30, shuffle=True)

base_estimator = DecisionTreeClassifier(max_depth=DEPTH)

ada = AdaBoostClassifier(estimator=base_estimator, n_estimators=NUMTREES)

# print(f'X_train head:\t{X_train.columns}\ny_train head:\t{y_train}\nX_test head:\t{X_test.columns}\ny_test head:\t{y_test}\n')
# Train the model
# ada.fit(Z_train, j_train)

In [99]:
# Lists to store elapsed time for each round
elapsed_times = []

# Timing the AdaBoost fitting process for each round
for stage, (X_train_stage, y_train_stage) in enumerate(zip(ada.staged_predict(Z_train), ada.staged_predict(j_train))):
    start_time = time.time()

    # Fit AdaBoost model for one round
    ada.fit(X_train_stage, y_train_stage)

    elapsed_time = time.time() - start_time
    elapsed_times.append(elapsed_time)

    print(f"Round {stage + 1} - Elapsed Time: {elapsed_time:.2f} seconds")


/Users/joedngangmeni/anaconda3/envs/TRF/lib/python3.11/site-packages/sklearn/base.py:458: UserWarning: X has feature names, but AdaBoostClassifier was fitted without feature names
  warnings.warn(


AttributeError: 'AdaBoostClassifier' object has no attribute 'n_classes_'

In [91]:
# f1_train, accuracy_train, precision_train, recall_train, mlogloss_train = clsBoostingStats(Z_train, j_train)
# f1_test, accuracy_test, precision_test, recall_test, mlogloss_test = clsBoostingStats(Z_test, j_test)


# print(f1_train)
# print(f1_test)
# print(mlogloss_train)
# print(mlogloss_test)

/Users/joedngangmeni/anaconda3/envs/TRF/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but AdaBoostClassifier was fitted with feature names
  warnings.warn(


InvalidParameterError: The 'y_pred' parameter of log_loss must be an array-like. Got <generator object AdaBoostClassifier.staged_predict_proba at 0x1785e6020> instead.